In [1]:
from mxESP32Debugger.debugger import Dbg
import time
import toml
import numpy as np
from loguru import logger

In [2]:
with open("gc6153.toml", 'r', encoding='utf-8') as f:
    config = toml.load(f)
    
init_reg_list = config['init']['reg_list']
mode_reg_list = config['mode']['reg_list']

In [3]:
dev = Dbg()
dev.usb_config(False)
dev.power_init()

2025-02-14 16:58:52.169 | INFO     | mxESP32Debugger.debugger:connect:248 - Using serial port to connect. Port: COM4
2025-02-14 16:58:52.170 | INFO     | mxESP32Debugger.debugger:connect:253 - Using ESP32-S3R8.
2025-02-14 16:58:52.171 | INFO     | mxESP32Debugger.debugger:connect:254 - Embedded software version: v1.0
2025-02-14 16:58:52.172 | INFO     | mxESP32Debugger.debugger:connect:255 - Library version: v241210.0.rc3.
2025-02-14 16:58:52.215 | INFO     | mxESP32Debugger.debugger:get_extboard_version:1652 - I2C config for extension board.
2025-02-14 16:58:52.238 | INFO     | mxESP32Debugger.debugger:get_extboard_version:1669 - Extension board version: v0.2.1


True

In [4]:
dev.i2c_config(sda_pullup=False, scl_pullup=False, freq=400_000)

(True, None)

In [5]:
dev.power_control("I2C", "3v3")
dev.power_control("SPI", "3v3")

True

In [6]:
pdn_pin = 39
mclk_en_pin = 40
dev.gpio_config(pin=pdn_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_config(pin=mclk_en_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_write_read(pin=mclk_en_pin, level=1)

(True, None)

In [7]:
time.sleep(100/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)

(True, None)

In [8]:
ret, chipid = dev.i2c_write_read(0x40, [0xf0], 2)
assert ret, "I2C read chipid failed"
logger.info("chipid: 0x{:04X}", (chipid[0] << 8) | chipid[1])

2025-02-14 16:58:52.611 | INFO     | __main__:<module>:3 - chipid: 0x6153


In [9]:
for reg in init_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write init reg list error"
for reg in mode_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write mode reg list error"
    

In [10]:
# configure spi as slave mode
dev.spi_config(miso_io_num=12, mosi_io_num=13, sclk_io_num=14, cs_io_num=15, mode=0, master_slave_mode=1)

(True, None)

In [11]:
ret, data = dev.spi_write_read([], 320*240, timeout=5)

TimeoutError: Data read timeout. No data received.